In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np

# Setup headless Chrome
options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

# Go to the page
url = 'https://www.tennisabstract.com/cgi-bin/player-more.cgi?p=104792/Gael-Monfils&table=mcp-serve'
driver.get(url)

# Wait for the page to load content (you can adjust this time or use WebDriverWait)
time.sleep(3)

# Get the rendered HTML
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# Now find the table
table = soup.find('table', id='mcp-serve')
print(table.prettify())

driver.quit()


<table border="0" cellpadding="4" cellspacing="0" class="tablesorter" id="mcp-serve">
 <thead>
  <tr>
   <th align="left">
    <span title="">
     Match
    </span>
   </th>
   <th align="left">
    <span title="">
     Result
    </span>
   </th>
   <th align="right" class="header">
    <span title="Percent of serves that
were unreturned">
     Unret%
    </span>
   </th>
   <th align="right" class="header">
    <span title="Percent of service points
won on either the serve or second shot">
     &lt;=3 W%
    </span>
   </th>
   <th align="right" class="header">
    <span title="Percent of points won when
return was put in play">
     RiP W%
    </span>
   </th>
   <th align="right" class="header">
    <span title="Serve Impact: Advanced stat
estimating how many service points
were won due to the serve">
     SvImpact
    </span>
   </th>
   <th align="right" class="header">
    <span title="Percent of first serves
that were unreturned">
     1st: Unret%
    </span>
   </th>
   <th a

In [2]:
data = []

for row in table.find_all('tr'):
    # Extract the text from each cell in the row
    cells = row.find_all('td')
    if cells:  # Check if the row has any cells
        for cell in cells:
            print(cell.get_text(strip=True))
            data.append((cell.get_text(strip=True)))


2025 Miami R32
W vsMunar
24.5%
37.3%
60.3%
33.3%
32.9%
49.3%
67.3%
46.0%
45.7%
47.4%
0.0%
9.4%
15.6%
48.3%
-29
2025 Miami R128
W vsMarozsan
34.9%
50.6%
54.9%
41.7%
44.4%
63.0%
60.0%
54.8%
43.3%
58.3%
33.3%
19.2%
30.8%
47.6%
72
2025 Indian Wells R32
L vsDimitrov
17.2%
27.0%
62.1%
22.9%
23.0%
35.1%
64.9%
32.3%
50.0%
47.2%
50.0%
9.5%
16.7%
57.9%
-26
2025 Dubai R32
L vsBerrettini
28.6%
39.7%
43.9%
33.5%
35.1%
48.6%
45.8%
43.5%
56.3%
57.1%
50.0%
22.7%
31.8%
41.2%
93
2025 Australian Open R16
L vsShelton
24.2%
41.1%
63.5%
35.2%
36.5%
62.2%
72.3%
55.0%
42.9%
54.3%
33.3%
7.3%
12.2%
52.6%
-52
2025 Australian Open R32
W vsFritz
37.2%
45.5%
66.7%
41.2%
48.6%
55.4%
65.8%
55.3%
31.4%
33.3%
100.0%
20.9%
32.6%
67.6%
50
2025 Auckland F
W vsBergs
26.7%
36.7%
70.0%
33.8%
39.4%
54.5%
80.0%
52.4%
62.5%
41.2%
25.0%
13.0%
17.4%
60.0%
4
2025 Auckland SF
W vsBasavareddy
33.0%
42.0%
50.9%
38.3%
45.8%
60.4%
53.8%
55.6%
57.1%
48.1%
100.0%
18.4%
21.1%
48.4%
-3
2025 Auckland QF
W vsDiaz Acosta
30.8%
50.0%
54.3%
39.

In [3]:
data = [d.replace('\xa0',' ') for d in data]

In [4]:
year_tourney_round = data[::17] # year tournament round in a single string

In [5]:
df_rows = []
for i in range(0, len(data)):
    if i % 17 == 0:
        row = data[i:i+17]
        df_rows.append(row)

In [6]:
cols = ['year tourney round', 'result opponent', 'overall: Unret%', 'overall: <=3 W%', 'overall: RiP W%', 'overall: SvImpact', '1st: Unret%',
        '1st: <=3 W%', '1st: RiP W%', '1st: SvImpact', '1st: D Wide%', '1st: A Wide%', '1st: A BP Wide%', '2nd: Unret%', '2nd: <=3 W%',
        '2nd: RiP W%', '2ndAgg']
df = pd.DataFrame(df_rows, columns=cols)
df['year'] = df['year tourney round'].apply(lambda x: x.split(' ')[0])
df['tourney'] = df['year tourney round'].apply(lambda x: (' ').join(x.split(' ')[1:-1]))
df['round'] = df['year tourney round'].apply(lambda x: x.split(' ')[-1]) 
df['result'] = df['result opponent'].apply(lambda x: x.split(' ')[0])
df['opponent'] = df['result opponent'].apply(lambda x: x.split(' ')[1][2:])
df.drop(columns=['year tourney round', 'result opponent'], inplace=True)
df.head(10)

,overall: Unret%,overall: <=3 W%,overall: RiP W%,overall: SvImpact,1st: Unret%,1st: <=3 W%,1st: RiP W%,1st: SvImpact,1st: D Wide%,1st: A Wide%,1st: A BP Wide%,2nd: Unret%,2nd: <=3 W%,2nd: RiP W%,2ndAgg,year,tourney,round,result,opponent
0,24.5%,37.3%,60.3%,33.3%,32.9%,49.3%,67.3%,46.0%,45.7%,47.4%,0.0%,9.4%,15.6%,48.3%,-29,2025,Miami,R32,W,Munar
1,34.9%,50.6%,54.9%,41.7%,44.4%,63.0%,60.0%,54.8%,43.3%,58.3%,33.3%,19.2%,30.8%,47.6%,72,2025,Miami,R128,W,Marozsan
2,17.2%,27.0%,62.1%,22.9%,23.0%,35.1%,64.9%,32.3%,50.0%,47.2%,50.0%,9.5%,16.7%,57.9%,-26,2025,Indian Wells,R32,L,Dimitrov
3,28.6%,39.7%,43.9%,33.5%,35.1%,48.6%,45.8%,43.5%,56.3%,57.1%,50.0%,22.7%,31.8%,41.2%,93,2025,Dubai,R32,L,Berrettini
4,24.2%,41.1%,63.5%,35.2%,36.5%,62.2%,72.3%,55.0%,42.9%,54.3%,33.3%,7.3%,12.2%,52.6%,-52,2025,Australian Open,R16,L,Shelton
5,37.2%,45.5%,66.7%,41.2%,48.6%,55.4%,65.8%,55.3%,31.4%,33.3%,100.0%,20.9%,32.6%,67.6%,50,2025,Australian Open,R32,W,Fritz
6,26.7%,36.7%,70.0%,33.8%,39.4%,54.5%,80.0%,52.4%,62.5%,41.2%,25.0%,13.0%,17.4%,60.0%,4,2025,Auckland,F,W,Bergs
7,33.0%,42.0%,50.9%,38.3%,45.8%,60.4%,53.8%,55.6%,57.1%,48.1%,100.0%,18.4%,21.1%,48.4%,-3,2025,Auckland,SF,W,Basavareddy
8,30.8%,50.0%,54.3%,39.8%,45.5%,66.7%,72.2%,59.7%,63.2%,42.9%,0.0%,5.6%,22.2%,35.3%,-60,2025,Auckland,QF,W,Diaz
9,41.9%,58.1%,65.6%,50.2%,52.4%,73.8%,80.0%,64.5%,46.7%,55.6%,100.0%,25.0%,31.3%,41.7%,102,2025,Auckland,R16,W,Struff


In [7]:
neworder = ['year','tourney', 'round', 'result', 'opponent', 'overall: Unret%', 'overall: <=3 W%', 'overall: RiP W%', 'overall: SvImpact', '1st: Unret%', '1st: <=3 W%',
       '1st: RiP W%', '1st: SvImpact', '1st: D Wide%', '1st: A Wide%', '1st: A BP Wide%',
       '2nd: Unret%', '2nd: <=3 W%', '2nd: RiP W%', '2ndAgg']

df = df[neworder]

In [8]:

df.to_csv('../data/monfils_serve_chart.csv', index=False) #save to csv in data folder

In [9]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

# Go to the page
url = 'https://www.tennisabstract.com/cgi-bin/player-more.cgi?p=104792/Gael-Monfils&table=serve-speed'
driver.get(url)

# Wait for the page to load content (you can adjust this time or use WebDriverWait)
time.sleep(3)

# Get the rendered HTML
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# Now find the table
table = soup.find('table', id='serve-speed')
print(table.prettify())

driver.quit()

<table border="0" cellpadding="4" cellspacing="0" class="tablesorter" id="serve-speed">
 <thead>
  <tr>
   <th align="left">
    Match
   </th>
   <th align="left">
    Result
   </th>
   <th align="right" class="header">
    <span title="Average serve speed">
     Avg Speed
    </span>
   </th>
   <th align="right" class="header">
    <span title="Average 1st serve speed">
     1st Avg
    </span>
   </th>
   <th align="right" class="header">
    <span title="Standard deviation of
1st serve speed">
     1st StDev
    </span>
   </th>
   <th align="right" class="header">
    <span t""="" title="Average speed of 1st serves
down the ">
     1st T Avg
    </span>
   </th>
   <th align="right" class="header">
    <span title="Average speed of
1st serves hit wide">
     1st Wide Avg
    </span>
   </th>
   <th align="right" class="header">
    <span title="Fastest second serve">
     Max 1st
    </span>
   </th>
   <th align="right" class="header">
    <span title="Slowest second serve">
  

In [10]:
speed = []

for row in table.find_all('tr'):
    # Extract the text from each cell in the row
    cells = row.find_all('td')
    if cells:  # Check if the row has any cells
        for cell in cells:
            print(cell.get_text(strip=True))
            speed.append((cell.get_text(strip=True)))


2024 US Open R64
L vsRuud
107.7
115.5
11.2
116.8
110.5
133
94
97.7
7.1
101.5
101.7
116
85
2024 US Open R128
W vsSchwartzman
103.9
111.2
11.4
115.4
107.5
128
79
94.4
7.2
98.6
95.8
107
80
2024 Wimbledon R32
L vsDimitrov
110.9
117.1
9.3
120.8
111.5
131
98
100.2
7.6
108.3
102.0
124
88
2024 Wimbledon R64
W vsWawrinka
109.4
114.4
11.2
118.4
105.6
132
91
100.9
6.8
106.6
91.0
114
88
2024 Wimbledon R128
W vsMannarino
107.4
113.5
9.4
117.1
109.9
129
93
96.2
7.3
93.7
97.0
119
82
2023 US Open R64
L vsRublev
105.3
112.9
11.3
117.5
108.3
136
85
95.9
5.7
100.0
97.4
105
83
2023 US Open R128
W vsDaniel
108.0
113.3
11.7
119.0
107.2
135
94
96.6
5.7
97.5
98.3
116
88
2021 US Open R32
L vsSinner
108.2
115.3
9.5
117.0
111.6
129
94
96.6
5.4
101.7
97.7
108
84
2021 US Open R64
W vsJohnson
106.4
112.6
12.3
111.8
108.7
130
65
97.1
6.1
98.8
95.3
113
83
2021 US Open R128
W vsCoria
109.9
117.9
9.9
119.3
112.9
135
99
96.3
5.5
102.3
91.3
106
87
2021 Wimbledon R128
W vsOconnell
110.5
115.8
9.7
118.1
109.4
132
91
99.1
8

In [11]:
speed = [s.replace('\xa0',' ') for s in speed]

In [12]:
year_tourney_round = speed[::15]

In [13]:
speed_df_rows = []
for i in range(0, len(speed)):
    if i % 15 == 0:
        row = speed[i:i+15]
        speed_df_rows.append(row)

In [14]:
cols = ['year tourney round', 'result opponent', 'avg speed', '1st: avg', '1st: stdev', '1st: t avg', '1st: wide avg', '1st: max', 
     '1st: min', '2nd: avg', '2nd: stdev', '2nd: t avg', '2nd: wide avg', '2nd: max', '2nd: min']

speed_df = pd.DataFrame(speed_df_rows, columns=cols)
speed_df['year'] = speed_df['year tourney round'].apply(lambda x: x.split(' ')[0])
speed_df['tourney'] = speed_df['year tourney round'].apply(lambda x: (' ').join(x.split(' ')[1:-1]))
speed_df['round'] = speed_df['year tourney round'].apply(lambda x: x.split(' ')[-1])
speed_df['result'] = speed_df['result opponent'].apply(lambda x: x.split(' ')[0])
speed_df['opponent'] = speed_df['result opponent'].apply(lambda x: x.split(' ')[1][2:])
speed_df.drop(columns=['year tourney round', 'result opponent'], inplace=True)
speed_df.head(10)

,avg speed,1st: avg,1st: stdev,1st: t avg,1st: wide avg,1st: max,1st: min,2nd: avg,2nd: stdev,2nd: t avg,2nd: wide avg,2nd: max,2nd: min,year,tourney,round,result,opponent
0,107.7,115.5,11.2,116.8,110.5,133,94,97.7,7.1,101.5,101.7,116,85,2024,US Open,R64,L,Ruud
1,103.9,111.2,11.4,115.4,107.5,128,79,94.4,7.2,98.6,95.8,107,80,2024,US Open,R128,W,Schwartzman
2,110.9,117.1,9.3,120.8,111.5,131,98,100.2,7.6,108.3,102.0,124,88,2024,Wimbledon,R32,L,Dimitrov
3,109.4,114.4,11.2,118.4,105.6,132,91,100.9,6.8,106.6,91.0,114,88,2024,Wimbledon,R64,W,Wawrinka
4,107.4,113.5,9.4,117.1,109.9,129,93,96.2,7.3,93.7,97.0,119,82,2024,Wimbledon,R128,W,Mannarino
5,105.3,112.9,11.3,117.5,108.3,136,85,95.9,5.7,100.0,97.4,105,83,2023,US Open,R64,L,Rublev
6,108.0,113.3,11.7,119.0,107.2,135,94,96.6,5.7,97.5,98.3,116,88,2023,US Open,R128,W,Daniel
7,108.2,115.3,9.5,117.0,111.6,129,94,96.6,5.4,101.7,97.7,108,84,2021,US Open,R32,L,Sinner
8,106.4,112.6,12.3,111.8,108.7,130,65,97.1,6.1,98.8,95.3,113,83,2021,US Open,R64,W,Johnson
9,109.9,117.9,9.9,119.3,112.9,135,99,96.3,5.5,102.3,91.3,106,87,2021,US Open,R128,W,Coria


In [15]:
neworder = ['year','tourney', 'round', 'result', 'opponent'] + cols[2:]
speed_df = speed_df[neworder]


In [16]:
for col in speed_df.columns[5:]:
    speed_df[col] = speed_df[col].replace('', np.nan)
speed_df

,year,tourney,round,result,opponent,avg speed,1st: avg,1st: stdev,1st: t avg,1st: wide avg,1st: max,1st: min,2nd: avg,2nd: stdev,2nd: t avg,2nd: wide avg,2nd: max,2nd: min
0,2024,US Open,R64,L,Ruud,107.7,115.5,11.2,116.8,110.5,133,94,97.7,7.1,101.5,101.7,116,85
1,2024,US Open,R128,W,Schwartzman,103.9,111.2,11.4,115.4,107.5,128,79,94.4,7.2,98.6,95.8,107,80
2,2024,Wimbledon,R32,L,Dimitrov,110.9,117.1,9.3,120.8,111.5,131,98,100.2,7.6,108.3,102.0,124,88
3,2024,Wimbledon,R64,W,Wawrinka,109.4,114.4,11.2,118.4,105.6,132,91,100.9,6.8,106.6,91.0,114,88
4,2024,Wimbledon,R128,W,Mannarino,107.4,113.5,9.4,117.1,109.9,129,93,96.2,7.3,93.7,97.0,119,82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2011,Roland Garros,R64,W,Rufin,109.7,118.7,10.1,NaN,NaN,133,94,99.0,4.7,NaN,NaN,111,92
96,2011,Roland Garros,R128,W,Phau,103.9,110.7,12.5,NaN,NaN,128,78,95.7,8.2,NaN,NaN,108,74
97,2011,Australian Open,R32,L,Wawrinka,103.6,110.2,11.1,NaN,NaN,131,82,93.5,6.3,NaN,NaN,108,79
98,2011,Australian Open,R64,W,Gil,105.9,111.0,8.6,NaN,NaN,127,92,93.9,9.8,NaN,NaN,131,82


In [17]:
for col in speed_df.columns[5:]:
    speed_df[col] = speed_df[col].astype(float)

In [18]:
speed_df.to_csv('../data/monfils_serve_speed.csv', index=False) #change the name of the file to save it as you want